In [ ]:
import pandas as pd
from tqdm import tqdm
from pykospacing import Spacing

import re
import torch
import rhinoMorph
from transformers import ElectraModel, ElectraTokenizer

tqdm.pandas()                  # tqdm과 pandas 통합

rn = rhinoMorph.startRhino()   # Rhino 객체 초기화

In [ ]:
spacing = Spacing()

def preprocessing(text):
    text = spacing(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 불용어 리스트 가져오기
def get_stopwords():
    stopwords = set()
    with open('../../../../../data/stopwords-ko.txt', 'r', encoding='utf-8') as file:
        for line in file:
            stopwords.add(line.strip())
    return stopwords

# 형태소 분석 및 불용어 제거 함수
def morph_and_remove_stopwords(text, stopwords):
    text = preprocessing(text)
    morphs = rhinoMorph.onlyMorph_list(rn, text)
    meaningful_words = [word for word in morphs if word not in stopwords]
    return meaningful_words

In [ ]:
# 불용어 리스트 가져오기
stopwords = get_stopwords()

df = pd.read_excel('../../../../../data/filtered_news.xlsx')

# 형태소 분석 및 불용어 제거를 통해 'processed_feature' 열 생성
df['processed_feature'] = df['feature'].progress_apply(lambda x: morph_and_remove_stopwords(x, stopwords))

# 리스트를 문자열로 변환
df['processed_feature_str'] = df['processed_feature'].apply(lambda x: ' '.join(x))

In [ ]:
# KoELECTRA 토크나이저와 모델을 로드합니다.
tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
model = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator')

# 텍스트의 임베딩을 얻는 함수
def get_koelectra_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()


# 각 처리된 피처에 대해 함수를 적용합니다.
df['koelectra_embedding'] = df['processed_feature_str'].apply(get_koelectra_embeddings)

# 시각화를 위해 DataFrame으로 변환합니다.
koelectra_embeddings_df = pd.DataFrame(df['koelectra_embedding'].tolist())